<a href="https://colab.research.google.com/github/elangbijak4/blockchain-codes-use-generative-AI/blob/main/Rev1_Infrastruktur_Blockchain_Dengan_Implementasi_ERC721.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import hashlib
import time
import json

## Kelas Block

In [18]:
class Block:
    def __init__(self, index, previous_hash, timestamp, data, hash):
        self.index = index
        self.previous_hash = previous_hash
        self.timestamp = timestamp
        self.data = data
        self.hash = hash

    @staticmethod
    def calculate_hash(index, previous_hash, timestamp, data):
        value = f"{index}{previous_hash}{timestamp}{data}"
        return hashlib.sha256(value.encode('utf-8')).hexdigest()

    @staticmethod
    def create_genesis_block():
        return Block(0, "0", int(time.time()), "Genesis Block", Block.calculate_hash(0, "0", int(time.time()), "Genesis Block"))

    def to_dict(self):
        return {
            "index": self.index,
            "previous_hash": self.previous_hash,
            "timestamp": self.timestamp,
            "data": self.data,
            "hash": self.hash
        }

## Kelas Smart Contract

In [19]:
class SmartContract:
    def __init__(self, code):
        self.code = code
        self.storage = {}

    def execute(self, blockchain, **kwargs):
        local_context = {
            'blockchain': blockchain,
            'storage': self.storage
        }
        local_context.update(kwargs)
        exec(self.code, {}, local_context)
        self.storage = local_context['storage']

Implementasi ERC721 ke dalam Kelas ERC721

In [20]:
class ERC721:
    def __init__(self):
        self._token_owner = {}  # tokenId => owner
        self._owned_tokens = {}  # owner => list of tokenIds
        self._token_approvals = {}  # tokenId => approved address
        self._operator_approvals = {}  # owner => (operator => approved)

    def balance_of(self, owner):
        if owner in self._owned_tokens:
            return len(self._owned_tokens[owner])
        return 0

    def owner_of(self, tokenId):
        return self._token_owner.get(tokenId, None)

    def transfer_from(self, from_address, to_address, tokenId):
        if self._is_approved_or_owner(from_address, tokenId):
            self._transfer(from_address, to_address, tokenId)

    def approve(self, to, tokenId):
        owner = self.owner_of(tokenId)
        if owner:
            self._token_approvals[tokenId] = to

    def set_approval_for_all(self, owner, operator, approved):
        if owner in self._operator_approvals:
            self._operator_approvals[owner][operator] = approved
        else:
            self._operator_approvals[owner] = {operator: approved}

    def is_approved_for_all(self, owner, operator):
        return self._operator_approvals.get(owner, {}).get(operator, False)

    def _is_approved_or_owner(self, spender, tokenId):
        owner = self.owner_of(tokenId)
        return spender == owner or \
               self._token_approvals.get(tokenId) == spender or \
               self.is_approved_for_all(owner, spender)

    def _transfer(self, from_address, to_address, tokenId):
        # Remove token from the current owner's list
        if from_address in self._owned_tokens:
            self._owned_tokens[from_address].remove(tokenId)

        # Add token to the new owner's list
        if to_address in self._owned_tokens:
            self._owned_tokens[to_address].append(tokenId)
        else:
            self._owned_tokens[to_address] = [tokenId]

        # Change token owner
        self._token_owner[tokenId] = to_address

        # Clear approval
        if tokenId in self._token_approvals:
            del self._token_approvals[tokenId]

    def mint(self, to, tokenId):
        if tokenId not in self._token_owner:
            self._token_owner[tokenId] = to
            if to in self._owned_tokens:
                self._owned_tokens[to].append(tokenId)
            else:
                self._owned_tokens[to] = [tokenId]

    def burn(self, tokenId):
        owner = self.owner_of(tokenId)
        if owner:
            self._owned_tokens[owner].remove(tokenId)
            del self._token_owner[tokenId]
            if tokenId in self._token_approvals:
                del self._token_approvals[tokenId]


### Integrasi ERC721 dengan Blockchain

In [21]:
class Blockchain:
    def __init__(self):
        self.chain = [Block.create_genesis_block()]
        self.erc721 = ERC721()  # Tambahkan inisialisasi ERC721
        self.smart_contracts = {}  # Menyimpan alamat blok dan smart contract code

    def get_latest_block(self):
        return self.chain[-1]

    def add_block(self, data):
        latest_block = self.get_latest_block()
        new_index = latest_block.index + 1
        new_timestamp = int(time.time())
        new_hash = Block.calculate_hash(new_index, latest_block.hash, new_timestamp, data)
        new_block = Block(new_index, latest_block.hash, new_timestamp, data, new_hash)
        self.chain.append(new_block)
        return new_block

    def is_chain_valid(self):
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i-1]

            if current_block.hash != Block.calculate_hash(current_block.index, current_block.previous_hash, current_block.timestamp, current_block.data):
                return False

            if current_block.previous_hash != previous_block.hash:
                return False

        return True

    def mint_nft(self, to, tokenId):
        self.erc721.mint(to, tokenId)
        block = self.add_block(f"Minted NFT {tokenId} to {to}")
        return block.hash

    def transfer_nft(self, from_address, to_address, tokenId):
        self.erc721.transfer_from(from_address, to_address, tokenId)
        block = self.add_block(f"Transferred NFT {tokenId} from {from_address} to {to_address}")
        return block.hash

    def store_smart_contract(self, code):
        contract = SmartContract(code)
        block = self.add_block(f"Smart contract stored")
        self.smart_contracts[block.hash] = contract
        return block.hash

    def execute_smart_contract_from_block(self, block_hash, **kwargs):
        if block_hash in self.smart_contracts:
            contract = self.smart_contracts[block_hash]
            contract.execute(self, **kwargs)
        else:
            print("Smart contract not found in the specified block")

    def to_json(self):
        return json.dumps([block.to_dict() for block in self.chain], indent=4)


## Demo Pembuatan Smart Contract

In [22]:
if __name__ == "__main__":
    # Membuat blockchain baru
    my_blockchain = Blockchain()

    # Menambahkan blok baru
    my_blockchain.add_block("Block 1 Data")
    my_blockchain.add_block("Block 2 Data")

    print("Blockchain valid:", my_blockchain.is_chain_valid())

    # Membuat smart contract sederhana
    smart_contract_code = """
def contract_function(storage):
    storage['counter'] = storage.get('counter', 0) + 1
    print(f"Counter value: {storage['counter']}")

contract_function(storage)
"""

    # Menyimpan smart contract ke blockchain
    block_hash = my_blockchain.store_smart_contract(smart_contract_code)
    print(f"Smart contract stored in block with hash: {block_hash}")

    # Menjalankan smart contract dari blockchain
    my_blockchain.execute_smart_contract_from_block(block_hash)

    # Menjalankan kembali smart contract untuk melihat perubahan state
    my_blockchain.execute_smart_contract_from_block(block_hash)

    # Mencetak isi blockchain dalam format JSON
    print("Blockchain content in JSON:")
    print(my_blockchain.to_json())

Blockchain valid: True
Smart contract stored in block with hash: 71e50808364269860b7fd8d9e5aa5a6792f28a27d145bebe141da53e8d2a1593
Counter value: 1
Counter value: 2
Blockchain content in JSON:
[
    {
        "index": 0,
        "previous_hash": "0",
        "timestamp": 1716914920,
        "data": "Genesis Block",
        "hash": "24213dcb4ebffac9c8612b3c634127b0799eb863c51e88e84d38c817f10009c5"
    },
    {
        "index": 1,
        "previous_hash": "24213dcb4ebffac9c8612b3c634127b0799eb863c51e88e84d38c817f10009c5",
        "timestamp": 1716914920,
        "data": "Block 1 Data",
        "hash": "89124417509103534468cecc5aa14a1773e9ac023d0ca0b18af1a98d195175ed"
    },
    {
        "index": 2,
        "previous_hash": "89124417509103534468cecc5aa14a1773e9ac023d0ca0b18af1a98d195175ed",
        "timestamp": 1716914920,
        "data": "Block 2 Data",
        "hash": "3b3a9857d6a83e8963884ebe2f1921900bcdeea643addd09fb9757146b2b21db"
    },
    {
        "index": 3,
        "previous_ha

## Demo Penggunaan ERC-721

In [23]:
if __name__ == "__main__":
    # Membuat blockchain baru
    my_blockchain = Blockchain()

    # Mint NFT baru
    my_blockchain.mint_nft("0x123", 1)
    my_blockchain.mint_nft("0x456", 2)

    # Transfer NFT
    my_blockchain.transfer_nft("0x123", "0x789", 1)

    # Periksa kepemilikan NFT
    print("Owner of NFT 1:", my_blockchain.erc721.owner_of(1))
    print("Owner of NFT 2:", my_blockchain.erc721.owner_of(2))

    # Validasi blockchain
    print("Blockchain valid:", my_blockchain.is_chain_valid())

    # Menampilkan isi blockchain dalam format JSON
    print("Blockchain content in JSON:")
    print(my_blockchain.to_json())

Owner of NFT 1: 0x789
Owner of NFT 2: 0x456
Blockchain valid: True
Blockchain content in JSON:
[
    {
        "index": 0,
        "previous_hash": "0",
        "timestamp": 1716914990,
        "data": "Genesis Block",
        "hash": "2ac8a3da35e81f40967b898e19ad634a07a70e3149a08117192d0244be3ee4ec"
    },
    {
        "index": 1,
        "previous_hash": "2ac8a3da35e81f40967b898e19ad634a07a70e3149a08117192d0244be3ee4ec",
        "timestamp": 1716914990,
        "data": "Minted NFT 1 to 0x123",
        "hash": "dd003a3c9e2c8a0c25db2c14278bd9e60b0f2441f438a84c6535cf141f4438e2"
    },
    {
        "index": 2,
        "previous_hash": "dd003a3c9e2c8a0c25db2c14278bd9e60b0f2441f438a84c6535cf141f4438e2",
        "timestamp": 1716914990,
        "data": "Minted NFT 2 to 0x456",
        "hash": "a155185f6d6bb51da8f34c20dac6f12086077da87e4a9e4b9d3b3e58d8f1e47a"
    },
    {
        "index": 3,
        "previous_hash": "a155185f6d6bb51da8f34c20dac6f12086077da87e4a9e4b9d3b3e58d8f1e47a",
      